In [6]:
import os, pathlib;
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

directory = os.getcwd()
p = pathlib.Path(directory)
parentDir= str(p.parents[0])

filepath = parentDir #+ "/DataFiles/" 
csvName = "UTD_LP_File_1.csv"
pickleName = "UTD_LP_File_1.pick"
#df_csv = pd.read_csv(csvName) #filepath + 
df = pd.read_pickle(pickleName) #filepath +
#display(df.head())
#df.describe()

In [7]:
##any data analysis will require filtering for harmonic 1 only, harm2 and harm3 will skew analysis
df_harm1 = df[df['harmonic']==1]
df_harm1 = df_harm1[['gammaTuple', 'power', 'Pin', 'Pout', 'Gain', 'PAE', 'drainEff', 'r', 'x']].copy()

df.set_index(keys=['gammaTuple'], drop=False,inplace=True)
uniqGammas=df['gammaTuple'].unique().tolist()

#print(uniqGammas)

listGamDf = []

for gam in uniqGammas:
    gamDf = df_harm1.loc[df_harm1.gammaTuple==gam]
    gamDf.index = range(len(gamDf))
    listGamDf.append(gamDf)


listGamDfC = listGamDf.copy()
#print(listGamDf[-1])

##Trying interpolated-slicing at Pout = its 75th percentile value, i.e. Pout = 12

In [8]:
## working on trying to find an objective function that fits pout on x-axis and power indices on y-axis
from scipy.optimize import curve_fit
from scipy.interpolate import barycentric_interpolate
from matplotlib import pyplot as plt

def findErrorPercent(y, y_line):
    avgErrorPercent = 0
    totalErrorPercents = 0

    for true, predicted in zip(y, y_line):
        error = abs(predicted-true)
        errorPercent = (error/true)*100
        totalErrorPercents += errorPercent

    avgErrorPercent = totalErrorPercents/len(y)
    return avgErrorPercent

def objective_poly3(x, a, b, c, d):
	return a * x + b * x**2 + c * x**3 + d

def objective_poly2(x, a, b, c):
	return a * x + b * x**2 + c

def objective_linear(x, a, b):
	return a * x + b

listFunctions = [objective_linear, objective_poly2, objective_poly3]
functionIDs = [id(x) for x in listFunctions]
allErrors = []


for gamDf in listGamDfC:
    poutList = gamDf[['Pout']].to_numpy().transpose().tolist()[0]
    powerList = gamDf[['power']].to_numpy().transpose().tolist()[0]
    errorList = []
    for objective in listFunctions:
        x, y = poutList, powerList
        popt, _ = curve_fit(objective, x, y)
        x_line = np.array(sorted(x))
        #plt.scatter(x, y)

        if objective_linear == objective:
            a,b = popt
            #print('y = %.5f * x + %.5f' % (a, b))
            y_line = objective(x_line, a, b)
            linear_error = findErrorPercent(y, y_line)
            errorList.append(linear_error)
        elif objective_poly2 == objective:
            a,b,c = popt
            #print('y = %.5f * x + %.5f * x^2 + %.5f' % (a, b, c))
            y_line = objective(x_line, a, b, c)
            quad_error = findErrorPercent(y, y_line)
            errorList.append(quad_error)
        else:
            a, b, c, d = popt
            #print('y = %.5f * x + %.5f * x^2 + %.5f * x^3 + %.5f' % (a, b, c, d))
            y_line = objective(x_line, a, b, c, d)
            cubic_error = findErrorPercent(y, y_line)
            errorList.append(cubic_error)
        #plt.plot(x_line, y_line, '--', color='red')
        #plt.show()
    allErrors.append(errorList)

allErrors =  np.array(allErrors).transpose()

linear_errors = allErrors[0].tolist()
print('Average linear error fitting Pout and Power indices', sum(linear_errors)/len(linear_errors))
quad_errors = allErrors[1].tolist()
print('Average quadratic error fitting Pout and Power indices', sum(quad_errors)/len(quad_errors))
cubic_errors = allErrors[2].tolist()
print(cubic_errors[1])
print('Average cubic error fitting Pout and Power indices', sum(cubic_errors)/len(cubic_errors))

Average linear error fitting Pout and Power indices 12.319378642276762
Average quadratic error fitting Pout and Power indices 4.3359013425020425
0.8623183087147398
Average cubic error fitting Pout and Power indices 1.4220629503713753


In [9]:
'''
index = [(ind2 - ind1)*(val - lower)/(upper - lower)] + ind1
for sanity check
(7-6)((12-11.514738351588367)/(12.111547248846863-11.514738351588367))+6 = 6.81
'''

###Using linear interpolation, since cubic creates outliers. Need a strategy to used cubic's accuracy but curb outliers.
### one idea could involve filtering those points where the pout isn't within the range of it's pouts for the power range. 

FIXED_POUT = 12.0
dfOfLoadsAtPoutX = pd.DataFrame()

cols = listGamDf[-1].columns.to_list()
cols.remove('Pout')
cols.remove('gammaTuple')
print(cols)
import numpy as np

for i,gamDf in enumerate(listGamDf):
    calcDict = {}
    poutList = gamDf[['Pout']].to_numpy().transpose().tolist()[0]
    for col in cols:
        colVals = gamDf[[col]].to_numpy().transpose().tolist()[0]
        #calcVal = np.interp(FIXED_POUT, poutList, colVals)
        calcVal = barycentric_interpolate(poutList, colVals, FIXED_POUT)
        calcDict[col] = round(float(calcVal),6)
    calcDict['Pout'] = FIXED_POUT
    calcDict['gammaTuple'] = gamDf['gammaTuple'][0]
    dfOfLoadsAtPoutX = dfOfLoadsAtPoutX.append(calcDict, ignore_index=True)
    listGamDf[i] = gamDf.append(calcDict, ignore_index=True).sort_values(by=['power'],ignore_index=True)

print(listGamDf[1])
#dfOfLoadsAtPoutX = dfOfLoadsAtPoutX.sort_values(by=['gammaTuple'],ignore_index=True)
#print(dfOfLoadsAtPoutX)



['power', 'Pin', 'Gain', 'PAE', 'drainEff', 'r', 'x']


  gammaTuple      power        Pin       Pout       Gain         PAE  \
0       (2,) -16.827679 -31.650419  12.000000  43.650419  239.105351   
1       (2,)   1.000000  -0.276393  14.165755  14.442149   22.456467   
2       (2,)   2.000000   0.679620  15.046296  14.366676   26.610909   
3       (2,)   3.000000   1.613413  15.876645  14.263232   30.896169   
4       (2,)   4.000000   2.516785  16.644300  14.127516   35.045073   
5       (2,)   5.000000   3.388196  17.354401  13.966205   39.075257   
6       (2,)   6.000000   4.225680  17.990077  13.764397   42.840306   
7       (2,)   7.000000   5.027269  18.540410  13.513140   46.225211   
8       (2,)   8.000000   5.799251  19.001790  13.202539   49.052729   
9       (2,)   9.000000   6.540957  19.379825  12.838867   51.307304   

     drainEff         r         x  
0  240.744104 -0.124754 -0.126850  
1   23.294054 -0.173202 -0.100002  
2   27.621510 -0.173207 -0.099999  
3   32.098896 -0.173209 -0.100000  
4   36.454355 -0.173208 -0.

In [5]:
'''
listGamDfC[-1] = listGamDfC[-1].append(pd.Series([np.nan]), ignore_index=True)

listGamDfC[-1].iloc[-1, listGamDfC[-1].columns.get_loc('Pout')] = 12.0

listGamDfC[-1].sort_values(by=['Pout'],ignore_index=True, inplace=True)

listGamDfC[-1]['power'] =  listGamDfC[-1]['power'].apply(float)

listGamDfC[-1].interpolate(inplace=True)

listGamDfC[-1] = listGamDfC[-1].drop(columns=[0])

print(listGamDfC[-1])
'''
pass